In [1]:
import datetime
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm_notebook as tqdm
from random import shuffle


pickle_in = open("pickles/data.pickle","rb")
data_dict = pickle.load(pickle_in)
pickle_in = open("pickles/tokens.pickle","rb")
token_dict = pickle.load(pickle_in)
VOCAB_SIZE = len(token_dict)
PADDING_INDEX = 0
pickle_in = open("pickles/vectorized_data.pickle","rb")
vectorized_data = pickle.load(pickle_in)

In [2]:
indices = []
current_index = 0

def shuffle_total_dataset(datalist):
    # Returns a randomly shuffled copy of a datalist
    shuffled_datalist = []
    create_shuffled_indices(datalist)
    for i in indices:
        shuffled_datalist.append(datalist[i])
    return(shuffled_datalist)

def create_shuffled_indices(datalist):
    # Creates a list of shuffled indices, which get_next uses.
    # The data is not copied, so it is not as RAM-hungry as "shuffle_total_dataset"
    global indices
    global current_index
    indices = [i for i in range(len(datalist))]
    shuffle(indices)
    current_index = 0
    
def get_next(datalist,batch_size=1):
    # Returns the next batch using the (possibly shuffled) indices list.
    global current_index
    all_data = []
    if len(datalist) - batch_size < current_index:
        if current_index >= len(datalist):
            return(None)
        else:
            batch_size = len(datalist) - current_index
    
    maxlen = 0
    flat_unpadded_text = []
    label = []
    
    for i in range(batch_size):
        [txt,lbl] = datalist[indices[current_index + i]]
        flat_text = flatten_list(txt)
        if(len(flat_text) > maxlen):
            maxlen = len(flat_text)
        flat_unpadded_text.append(flat_text)
        label.append([lbl])
        
    current_index += batch_size
    
    text = []
    for unpadded_txt in flat_unpadded_text:
        text.append(pad(unpadded_txt,maxlen))

    return([text,label])
    
def pad(txt,length):
    # Adds padding to a text, so it meets the correct length requirement
    while(len(txt) < length):
        txt.append(PADDING_INDEX)
    return(txt)
    
def flatten_list(listoflists):
    # Flattens a list like [[x,y],[z]] to [x,y,z]
    resultinglist = []
    for elementarylist in listoflists:
        for element in elementarylist:
            resultinglist.append(element)
    return(resultinglist)

def get_sample(dataset,total_amount):
    # Returns a balanced random sample of the given dataset
    dataset = shuffle_total_dataset(dataset)
    sample = []
    half_amount = total_amount //2
    pos_amount = 0
    neg_amount = 0
    i = 0
    print("total len: ",len(dataset))
    while (pos_amount < half_amount) or (neg_amount < half_amount):
        [example,lbl] = dataset[i]
        if lbl == 0 and (neg_amount < half_amount):
            sample.append([example,lbl])
            neg_amount += 1
        if lbl == 1 and (pos_amount < half_amount):
            sample.append([example,lbl])
            pos_amount += 1
        i += 1
    return(sample)

def split_percentage(shuffled_dataset,devpct):
    shuffled_again = shuffle_total_dataset(shuffled_dataset)
    dev_amount = int(len(shuffled_again)*devpct)
    return([shuffled_again[dev_amount:],shuffled_again[:dev_amount]])

In [3]:
def split_kfold(dataset,k):
    # Returns a list containing k partitions of the dataset that are as close in size as possible.
    subset_length = len(dataset)/k
    if subset_length != len(dataset)//k:
        subset_length = subset_length + 1
    subset_length = int(subset_length)
    subsets = []
    if subset_length == 0:
        subsets = dataset
        while len(subsets) < k:
            subsets.append([])
    else:
        for k_i in range(k-1):
            subsets.append(dataset[subset_length*k_i:(subset_length*k_i)+subset_length])
        subsets.append(dataset[subset_length*(k-1):])

    return(subsets)
    
def get_split(subsets,i):
    # Returns the split where one of the subsets is used as devset, rest as trainset.
    total = len(subsets)
    actual_index = i - total * (i // total)
    dev_set = subsets[actual_index]
    train_set = []
    for j in range(total):
        if j != actual_index:
            for element in subsets[j]:
                train_set.append(element)
    return [train_set, dev_set]

In [4]:
class BinaryLSTMClassifier(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, output_size,layers=3):
        super(BinaryLSTMClassifier, self).__init__()
        
        # 1. Embedding Layer
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        
        # 2. LSTM Layer
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True, num_layers=layers)
        
        # 3. Dense Layer
        self.hidden2out = nn.Linear(hidden_dim, output_size)
        
        # Optional dropout layer
        self.dropout_layer = nn.Dropout(p=0.4)

    def forward(self, batch_text):
        embeddings = self.embeddings(batch_text)
        _, (ht, _) = self.lstm(embeddings)

        lstm_output = ht[-1]
        lstm_output = self.dropout_layer(lstm_output)
        final_output = self.hidden2out(lstm_output)
        return final_output

In [5]:
def epoch_metrics(predictions,correct,do_print=True):
    # Prints out the metrics of the given data, and returns f-score and accuracy
    predictions = flatten_list(predictions)
    unlisted_preds = []
    for elem in predictions:
        unlisted_preds.append(elem[0])
        
    correct = flatten_list(correct)
    unlisted_corrects = []
    for elem in correct:
        unlisted_corrects.append(elem[0])
    
    
    classified_preds = []
    for pred in unlisted_preds:
        if pred > 0.5:
            classified_preds.append(1)
        else:
            classified_preds.append(0)
    
    [tp, fn, fp, tn, relevant, nonrelevant] = [0,0,0,0,0,0]
    
    for current in range(len(unlisted_corrects)):
        if unlisted_corrects[current] == 1:
            relevant += 1
            if classified_preds[current] == 1:
                tp += 1
            else:
                fn += 1
        else:
            nonrelevant += 1
            if classified_preds[current] == 1:
                fp += 1
            else:
                tn += 1
                
    [p,r,f,a] = ["/","/",0,round((tp+tn)/(tp+fp+tn+fn),3)]
    
    if relevant != 0:
        r = (tp/relevant)

    if (tp+fp) != 0:
        p = (tp/(tp+fp))
        if (relevant != 0):
            if ((p+r) != 0):
                f =  round(2*(p*r)/(p+r),3)
            r = round(r,2)
        p = round(p,2)
            
    if do_print:
        print("precision: ",p," - recall: ",r," - accuracy: ",a," - f-score: ",f)
    
    return([a,f])

In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')

## If this being commented out is not a problem, remove it altogether
#def label_indices(vector):
#    idxs = []
#    for i in range(len(vector)):
#        if vector[i] == 1:
#            idxs.append(i)
#    return(idxs)

def train(model, train_iter, dev_iter, batch_size=1, max_epochs=15):
    # Train a model with the given parameters
    loss_function = nn.BCELoss()
    sigm= nn.Sigmoid()
    optimizer = optim.Adam(model.parameters())

    f_s = []
    a_s = []
    max_f = -1
    best_model = None
    for epoch in tqdm(range(max_epochs)):
        predictions, correct = [], []
        create_shuffled_indices(train_iter)
        print("=== Epoch ", epoch, " ===",datetime.datetime.now().strftime('%H:%M:%S'))
        model.train()
        for example in tqdm(range(1+(len(train_iter)//batch_size))):
            examples = get_next(train_iter, batch_size)
            if examples != None: #happens when train_iter//batch_size is integer
                optimizer.zero_grad()
                pred = model(torch.LongTensor(examples[0]).transpose(0,1).to(device))
                loss = loss_function(sigm(pred),torch.FloatTensor(examples[1]).to(device))
                loss.backward()
                optimizer.step()
                predictions.append(sigm(pred).data.tolist())
                correct.append(examples[1])
        
        print("Training results:")
        epoch_metrics(predictions,correct)
        predictions, correct = [], []
        create_shuffled_indices(dev_iter)
        model.eval()
        for example in range(1+(len(dev_iter)//batch_size)):
            examples = get_next(dev_iter, batch_size)
            if examples != None: #happens when train_iter//batch_size is integer
                pred = model(torch.LongTensor(examples[0]).transpose(0,1).to(device))
                predictions.append(sigm(pred).data.tolist())
                correct.append(examples[1])
            
        print("Development results:")
        [a,f] = epoch_metrics(predictions,correct)
        f_s.append(f)
        a_s.append(a)
        if max_f == -1:
            max_f = f
            best_model = model
        elif f >= max_f:
            print("################# NEW BEST #################")
            max_f = f
            best_model = model
            
        patience = 5
        if (epoch > patience):
            compare_against = f_s[epoch-patience]
            previous_f_s = []
            for i in range(patience):
                previous_f_s.append(f_s[epoch-i])
            
            bad = True
            for prevfs in previous_f_s:
                if prevfs > compare_against:
                    bad = False
                    break
            if bad:
                print("Not enough f-score improvement, stopping.")
                print(compare_against," is larger than ",previous_f_s)
                break
            
    return([f_s, a_s, best_model])           

In [7]:
def create_model(h,e,l,k,bs = 20,data_amount='all', no_kfold=False,devpct=0.1):
    # Create a model with the given parameters, starting with that batch size.
    # If the batch size is too large, smaller ones are attempted.
    # The models, as well as their metrics, are saved.
    shuffled_dataset = shuffle_total_dataset(vectorized_data["train"])
    if data_amount != 'all':
        shuffled_dataset = get_sample(shuffled_dataset,data_amount)
        
    if not(no_kfold):
        split_dataset = split_kfold(shuffled_dataset,k)
        
    for current_fold in range(k):
        if not(no_kfold):
            [training_set, development_set] = get_split(split_dataset,current_fold)
        else:
            [training_set, development_set] = split_percentage(shuffled_dataset,devpct)
            
        print("Try with BS: ",bs)

        while(True): # Keep trying with lower batch sizes
            try:
                print(f"H: {h}, E: {e}, L: {l}, CV_i: {current_fold}")
                classifier = BinaryLSTMClassifier(e, h, VOCAB_SIZE, 1,layers=l)  
                [f_scores, a_scores, best_model] = \
                    train(classifier.to(device), training_set, development_set, max_epochs=15, batch_size=bs)
                if data_amount != 'all':
                    pickle_out = open("models/CV-"+str(h)+"-"+str(e)+"-"+str(l)+"-"+str(data_amount)+"-"+str(current_fold)+".pickle","wb")
                else:
                    pickle_out = open("models/CV-"+str(h)+"-"+str(e)+"-"+str(l)+"-"+str(current_fold)+".pickle","wb")
                pickle.dump(best_model,pickle_out)
                pickle_out.close()
                if data_amount != 'all':
                    pickle_out = open("models/metrics/CV-metrics-"+str(h)+"-"+str(e)+"-"+str(l)+"-"+str(data_amount)+"-"+str(current_fold)+".pickle","wb")
                else:
                    pickle_out = open("models/metrics/CV-metrics-"+str(h)+"-"+str(e)+"-"+str(l)+"-"+str(current_fold)+".pickle","wb")
                pickle.dump([f_scores, a_scores], pickle_out)
                pickle_out.close()
                break # Do not try again when this batch size leads to a result
            except RuntimeError:
                #bs -= 1
                bs *= 0.9
                bs = int(bs)
                if bs <= 0:
                    print("Model too large, BS of 1 too big. Skipping")
                    break
                print("That didn't work. Try with BS: ",bs)
            
                
hidden_dims = [300]
embedding_dims = [300]
layer_amounts = [1]
fold_amount = [5]

hidden_dims = []
embedding_dims = []
layer_amounts = []
fold_amount = []

for h in hidden_dims:
    for e in embedding_dims:
        for l in layer_amounts:
            for k in fold_amount:
                create_model(h,e,l,k,bs=15)

In [8]:
def create_model(h,e,l,k,bs = 20,data_amount='all', no_kfold=False,devpct=0.1):
    # Create a model with the given parameters, starting with that batch size.
    # If the batch size is too large, smaller ones are attempted.
    # The models, as well as their metrics, are saved.
    shuffled_dataset = shuffle_total_dataset(vectorized_data["train"])
    if data_amount != 'all':
        shuffled_dataset = get_sample(shuffled_dataset,data_amount)
        
    if not(no_kfold):
        split_dataset = split_kfold(shuffled_dataset,k)
        
    for current_fold in range(k):
        if not(no_kfold):
            [training_set, development_set] = get_split(split_dataset,current_fold)
        else:
            [training_set, development_set] = split_percentage(shuffled_dataset,devpct)
            
        print("Try with BS: ",bs)

        while(True): # Keep trying with lower batch sizes
            try:
                if data_amount != 'all':
                    print(f"H: {h}, E: {e}, L: {l}, datasize: {data_amount}, CV_i: {current_fold}")
                else:
                    print(f"H: {h}, E: {e}, L: {l}, CV_i: {current_fold}")
                classifier = BinaryLSTMClassifier(e, h, VOCAB_SIZE, 1,layers=l)  
                [f_scores, a_scores, best_model] = \
                    train(classifier.to(device), training_set, development_set, max_epochs=15, batch_size=bs)
                if data_amount != 'all':
                    pickle_out = open("models/CV-"+str(h)+"-"+str(e)+"-"+str(l)+"-"+str(data_amount)+"-"+str(current_fold)+".pickle","wb")
                else:
                    pickle_out = open("models/CV-"+str(h)+"-"+str(e)+"-"+str(l)+"-"+str(current_fold)+".pickle","wb")
                pickle.dump(best_model,pickle_out)
                pickle_out.close()
                if data_amount != 'all':
                    pickle_out = open("models/metrics/CV-metrics-"+str(h)+"-"+str(e)+"-"+str(l)+"-"+str(data_amount)+"-"+str(current_fold)+".pickle","wb")
                else:
                    pickle_out = open("models/metrics/CV-metrics-"+str(h)+"-"+str(e)+"-"+str(l)+"-"+str(current_fold)+".pickle","wb")
                pickle.dump([f_scores, a_scores], pickle_out)
                pickle_out.close()
                break # Do not try again when this batch size leads to a result
            except RuntimeError:
                #bs -= 1
                bs *= 0.9
                bs = int(bs)
                if bs <= 0:
                    print("Model too large, BS of 1 too big. Skipping")
                    break
                print("That didn't work. Try with BS: ",bs)
                
                
# [50,100,150,500,1000,1500,5000,10000,15000]
data_amounts = [10000,15000]#
for data_size in data_amounts:
    create_model(300,300,2,5,bs=14,data_amount=data_size, no_kfold=True,devpct=0.1)
                

total len:  18854
Try with BS:  14
H: 300, E: 300, L: 2, datasize: 10000, CV_i: 0


=== Epoch  0  === 08:41:24


That didn't work. Try with BS:  12
H: 300, E: 300, L: 2, datasize: 10000, CV_i: 0


=== Epoch  0  === 08:41:50


Training results:
precision:  0.59  - recall:  0.59  - accuracy:  0.596  - f-score:  0.594
Development results:
precision:  0.59  - recall:  0.85  - accuracy:  0.611  - f-score:  0.695
=== Epoch  1  === 08:46:09


...

Training results:
precision:  0.99  - recall:  0.99  - accuracy:  0.99  - f-score:  0.99
Development results:
precision:  0.88  - recall:  0.86  - accuracy:  0.879  - f-score:  0.872
=== Epoch  6  === 18:10:16


Training results:
precision:  0.99  - recall:  0.99  - accuracy:  0.994  - f-score:  0.994
Development results:
precision:  0.8  - recall:  0.95  - accuracy:  0.865  - f-score:  0.869
=== Epoch  7  === 18:16:49


Training results:
precision:  0.99  - recall:  0.99  - accuracy:  0.993  - f-score:  0.993
Development results:
precision:  0.73  - recall:  0.97  - accuracy:  0.812  - f-score:  0.831
=== Epoch  8  === 18:23:20


Training results:
precision:  0.99  - recall:  0.99  - accuracy:  0.995  - f-score:  0.995
Development results:
precision:  0.86  - recall:  0.88  - accuracy:  0.874  - f-score:  0.869
=== Epoch  9  === 18:29:52


Training results:
precision:  1.0  - recall:  1.0  - accuracy:  0.997  - f-score:  0.997
Development results:
precision:  0.86  - recall:  0.89  - accuracy:  0.883  - f-score:  0.878
Not enough f-score improvement, stopping.
0.883  is larger than  [0.878, 0.869, 0.831, 0.869, 0.872]
